# Face Recognition using VGG Face Other Architecture

Source: https://github.com/rcmalli/keras-vggface

## Initialize Two Model

In [1]:
from keras_vggface.vggface import VGGFace
from keras_vggface import utils
from keras.preprocessing.image import load_img, img_to_array
import numpy as np

/home/agusgun/anaconda3/envs/basic/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
vgg_features_vgg16 = VGGFace(model='vgg16', include_top=False, input_shape=(224, 224, 3), pooling='avg')
vgg_features_resnet50 = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

## Feature Extraction

In [3]:
def preprocess_image(img_path, version=1):
    img = load_img(img_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = utils.preprocess_input(img, version=version)
    return img

In [4]:
%%time
img_representation = vgg_features_vgg16.predict(preprocess_image('../face_data/1_0.png'))

CPU times: user 1.12 s, sys: 43.4 ms, total: 1.17 s
Wall time: 367 ms


In [5]:
img_representation[0].shape

(512,)

In [6]:
%%time
img_representation = vgg_features_resnet50.predict(preprocess_image('../face_data/1_0.png'))

CPU times: user 1.96 s, sys: 429 ms, total: 2.39 s
Wall time: 1.48 s


In [7]:
img_representation[0].shape

(2048,)

## Testing Using Own Dataset

In [8]:
import os

In [9]:
img_path_list = os.listdir('../face_data')
img_path_list = [os.path.join('../face_data', img_path) for img_path in img_path_list]

In [10]:
target = []
for img_path in img_path_list:
    if ('/1_' in img_path):
        target.append(1)
    elif ('/2_' in img_path):
        target.append(2)
    elif ('/7_' in img_path):
        target.append(3)
    elif ('/12_' in img_path):
        target.append(3)
    elif ('/17_' in img_path):
        target.append(4)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train_img_path_list, X_test_img_path_list, y_train, y_test = train_test_split(
    img_path_list, target, test_size=0.2, random_state=0
)

In [13]:
def extract_batch(img_path_list, extractor, version):
    result = []
    for img_path in img_path_list:
        result.append(extractor.predict(preprocess_image(img_path, version))[0])
    return result

In [14]:
from scipy.spatial.distance import euclidean

In [15]:
def predict(image_representation_list, true_label, test_representation):
    minimum_label = None
    minimum_distance = np.inf
    
    for idx, image_representation in enumerate(image_representation_list):
        distance = euclidean(image_representation, test_representation)
        if distance < minimum_distance:
            minimum_distance = distance
            minimum_label = true_label[idx]
    
    return minimum_label

### First Model

In [16]:
%%time
X_train_representation_vgg16 = extract_batch(X_train_img_path_list, vgg_features_vgg16, 1)
X_test_representation_vgg16 = extract_batch(X_test_img_path_list, vgg_features_vgg16, 1)

CPU times: user 4min 31s, sys: 16.3 s, total: 4min 47s
Wall time: 44.7 s


In [17]:
predict(X_train_representation_vgg16, y_train, X_test_representation_vgg16[0])

4

In [18]:
predicted_list = []
for test_representation in X_test_representation_vgg16:
    predicted_list.append(predict(X_train_representation_vgg16, y_train, test_representation))

In [19]:
from sklearn.metrics import accuracy_score, f1_score

In [20]:
print(accuracy_score(predicted_list, y_test))
print(f1_score(predicted_list, y_test, average='weighted'))

0.9824561403508771
0.9821776663881926


### Second Model

In [21]:
%%time
X_train_representation_resnet50 = extract_batch(X_train_img_path_list, vgg_features_resnet50, 2)
X_test_representation_resnet50 = extract_batch(X_test_img_path_list, vgg_features_resnet50, 2)

CPU times: user 3min 36s, sys: 1min 10s, total: 4min 46s
Wall time: 39.3 s


In [22]:
predict(X_train_representation_resnet50, y_train, X_test_representation_resnet50[0])

4

In [23]:
predicted_list = []
for test_representation in X_test_representation_resnet50:
    predicted_list.append(predict(X_train_representation_resnet50, y_train, test_representation))

In [24]:
print(accuracy_score(predicted_list, y_test))
print(f1_score(predicted_list, y_test, average='weighted'))

0.9824561403508771
0.9820793814601864
